## Install dependencies

In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
from pymongo import MongoClient
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import year
import time

In [8]:
# Add a decorator to time the function
def timeit(method):
    def timed(*args, **kw):
        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()
        print(f"{method.__name__} took: {end_time - start_time} sec")
        return result
    return timed

## Clickhouse vs Postgre in data loading

### Read data from Clickhouse

In [9]:
@timeit
def connect_clickhouse():

    packages = [
        "com.clickhouse.spark:clickhouse-spark-runtime-3.4_2.12:0.8.0",
        "com.clickhouse:clickhouse-client:0.7.0",
        "com.clickhouse:clickhouse-http-client:0.7.0",
        "org.apache.httpcomponents.client5:httpclient5:5.2.1"

    ]

    spark = (SparkSession.builder
            .config("spark.jars.packages", ",".join(packages))
            .getOrCreate())

    # register the clickhouse catalog
    # this makes us be able to work with multiple clickhouse databases and tables
    spark.conf.set("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
    spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
    spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
    spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
    spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
    spark.conf.set("spark.sql.catalog.clickhouse.password", "")
    spark.conf.set("spark.sql.catalog.clickhouse.database", "movie_lens")
    spark.conf.set("spark.clickhouse.write.format", "json")
    
    return spark

spark = connect_clickhouse()

connect_clickhouse took: 0.13023877143859863 sec


In [10]:
@timeit
def read_movies_from_clickhouse(spark):
    df = spark.sql("select * from clickhouse.movie_lens.movies")
    df.show(20)
    
read_movies_from_clickhouse(spark)

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
|       5|Father of the Bri...|              Comedy|
|       6|         Heat (1995)|Action|Crime|Thri...|
|       7|      Sabrina (1995)|      Comedy|Romance|
|       8| Tom and Huck (1995)|  Adventure|Children|
|       9| Sudden Death (1995)|              Action|
|      10|    GoldenEye (1995)|Action|Adventure|...|
|      11|American Presiden...|Comedy|Drama|Romance|
|      12|Dracula: Dead and...|       Comedy|Horror|
|      13|        Balto (1995)|Adventure|Animati...|
|      14|        Nixon (1995)|               Drama|
|      15|Cutthroat Island ...|Action|Adventure|...|
|      16|       Casino (1995)|         Crime|

In [11]:
@timeit
def read_ratings_from_clickhouse(spark):
    df = spark.sql("select * from clickhouse.movie_lens.ratings")
    df.show(20)
    
read_ratings_from_clickhouse(spark)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|      1|      17|   4.0|1999-12-04 02:24:37|
|      1|      25|   1.0|1999-12-04 02:43:48|
|      1|      29|   2.0|1999-11-22 07:36:16|
|      1|      30|   5.0|1999-12-04 02:24:37|
|      1|      32|   5.0|1999-11-22 07:00:58|
|      1|      34|   2.0|1999-11-22 06:54:51|
|      1|      36|   1.0|1999-12-04 02:23:28|
|      1|      80|   5.0|1999-12-04 02:22:23|
|      1|     110|   3.0|1999-11-22 07:38:39|
|      1|     111|   5.0|1999-12-04 02:23:28|
|      1|     161|   1.0|1999-11-22 07:39:22|
|      1|     166|   5.0|1999-11-22 06:54:02|
|      1|     176|   4.0|1999-12-02 03:18:16|
|      1|     223|   3.0|1999-12-02 04:13:30|
|      1|     232|   5.0|1999-11-22 06:54:02|
|      1|     260|   5.0|1999-11-22 06:58:16|
|      1|     302|   4.0|1999-12-04 03:34:32|
|      1|     306|   5.0|1999-12-04 02:21:28|
|      1|     307|   5.0|1999-12-0

### Query data from Clickhouse

In [12]:
@timeit
def show_head_of_ratings(spark, n_rows=20):
    df = spark.sql(f"select * from clickhouse.movie_lens.ratings limit {n_rows}")
    df.show()
    
show_head_of_ratings(spark)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|      1|      17|   4.0|1999-12-04 02:24:37|
|      1|      25|   1.0|1999-12-04 02:43:48|
|      1|      29|   2.0|1999-11-22 07:36:16|
|      1|      30|   5.0|1999-12-04 02:24:37|
|      1|      32|   5.0|1999-11-22 07:00:58|
|      1|      34|   2.0|1999-11-22 06:54:51|
|      1|      36|   1.0|1999-12-04 02:23:28|
|      1|      80|   5.0|1999-12-04 02:22:23|
|      1|     110|   3.0|1999-11-22 07:38:39|
|      1|     111|   5.0|1999-12-04 02:23:28|
|      1|     161|   1.0|1999-11-22 07:39:22|
|      1|     166|   5.0|1999-11-22 06:54:02|
|      1|     176|   4.0|1999-12-02 03:18:16|
|      1|     223|   3.0|1999-12-02 04:13:30|
|      1|     232|   5.0|1999-11-22 06:54:02|
|      1|     260|   5.0|1999-11-22 06:58:16|
|      1|     302|   4.0|1999-12-04 03:34:32|
|      1|     306|   5.0|1999-12-04 02:21:28|
|      1|     307|   5.0|1999-12-0

In [13]:
show_head_of_ratings(spark, 1000000)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|      1|      17|   4.0|1999-12-04 02:24:37|
|      1|      25|   1.0|1999-12-04 02:43:48|
|      1|      29|   2.0|1999-11-22 07:36:16|
|      1|      30|   5.0|1999-12-04 02:24:37|
|      1|      32|   5.0|1999-11-22 07:00:58|
|      1|      34|   2.0|1999-11-22 06:54:51|
|      1|      36|   1.0|1999-12-04 02:23:28|
|      1|      80|   5.0|1999-12-04 02:22:23|
|      1|     110|   3.0|1999-11-22 07:38:39|
|      1|     111|   5.0|1999-12-04 02:23:28|
|      1|     161|   1.0|1999-11-22 07:39:22|
|      1|     166|   5.0|1999-11-22 06:54:02|
|      1|     176|   4.0|1999-12-02 03:18:16|
|      1|     223|   3.0|1999-12-02 04:13:30|
|      1|     232|   5.0|1999-11-22 06:54:02|
|      1|     260|   5.0|1999-11-22 06:58:16|
|      1|     302|   4.0|1999-12-04 03:34:32|
|      1|     306|   5.0|1999-12-04 02:21:28|
|      1|     307|   5.0|1999-12-0

In [14]:
show_head_of_ratings(spark, 7000000)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|      1|      17|   4.0|1999-12-04 02:24:37|
|      1|      25|   1.0|1999-12-04 02:43:48|
|      1|      29|   2.0|1999-11-22 07:36:16|
|      1|      30|   5.0|1999-12-04 02:24:37|
|      1|      32|   5.0|1999-11-22 07:00:58|
|      1|      34|   2.0|1999-11-22 06:54:51|
|      1|      36|   1.0|1999-12-04 02:23:28|
|      1|      80|   5.0|1999-12-04 02:22:23|
|      1|     110|   3.0|1999-11-22 07:38:39|
|      1|     111|   5.0|1999-12-04 02:23:28|
|      1|     161|   1.0|1999-11-22 07:39:22|
|      1|     166|   5.0|1999-11-22 06:54:02|
|      1|     176|   4.0|1999-12-02 03:18:16|
|      1|     223|   3.0|1999-12-02 04:13:30|
|      1|     232|   5.0|1999-11-22 06:54:02|
|      1|     260|   5.0|1999-11-22 06:58:16|
|      1|     302|   4.0|1999-12-04 03:34:32|
|      1|     306|   5.0|1999-12-04 02:21:28|
|      1|     307|   5.0|1999-12-0

In [15]:
@timeit
def get_ratings_by_user(spark, user_id):
    df = spark.sql(f"select * from clickhouse.movie_lens.ratings where user_id = {user_id}")
    df.show()
    return df

df = get_ratings_by_user(spark, 1)
df.count()

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|      1|      17|   4.0|1999-12-04 02:24:37|
|      1|      25|   1.0|1999-12-04 02:43:48|
|      1|      29|   2.0|1999-11-22 07:36:16|
|      1|      30|   5.0|1999-12-04 02:24:37|
|      1|      32|   5.0|1999-11-22 07:00:58|
|      1|      34|   2.0|1999-11-22 06:54:51|
|      1|      36|   1.0|1999-12-04 02:23:28|
|      1|      80|   5.0|1999-12-04 02:22:23|
|      1|     110|   3.0|1999-11-22 07:38:39|
|      1|     111|   5.0|1999-12-04 02:23:28|
|      1|     161|   1.0|1999-11-22 07:39:22|
|      1|     166|   5.0|1999-11-22 06:54:02|
|      1|     176|   4.0|1999-12-02 03:18:16|
|      1|     223|   3.0|1999-12-02 04:13:30|
|      1|     232|   5.0|1999-11-22 06:54:02|
|      1|     260|   5.0|1999-11-22 06:58:16|
|      1|     302|   4.0|1999-12-04 03:34:32|
|      1|     306|   5.0|1999-12-04 02:21:28|
|      1|     307|   5.0|1999-12-0

141

In [16]:
@timeit
def get_ratings_by_movies(spark, movie_id):
    df = spark.sql(f"select * from clickhouse.movie_lens.ratings where movie_id = {movie_id}")
    df.show()
    return df

df = get_ratings_by_movies(spark, 1)
df.count()

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|     10|       1|   2.5|2007-01-20 10:53:51|
|     11|       1|   3.0|1996-12-09 05:44:36|
|     17|       1|   4.0|2002-07-22 09:42:31|
|     19|       1|   3.0|2000-11-20 14:14:48|
|     20|       1|   5.0|2019-03-21 23:03:50|
|     23|       1|   3.0|2019-10-04 18:52:12|
|     24|       1|   4.0|1999-12-16 21:15:45|
|     28|       1|   4.0|2000-06-20 01:08:47|
|     33|       1|   5.0|2008-11-11 07:07:49|
|     34|       1|   4.0|2007-11-11 07:43:38|
|     36|       1|   3.0|2015-01-10 04:42:19|
|     37|       1|   1.0|2001-06-17 04:49:45|
|     43|       1|   5.0|1996-06-09 19:14:09|
|     46|       1|   4.0|2015-10-04 16:36:55|
|     51|       1|   3.5|2006-06-27 02:14:04|
|     54|       1|   4.0|1997-05-20 18:36:51|
|     57|       1|   4.0|1997-01-07 03:07:11|
|     59|       1|   4.0|2002-08-27 03:57:42|
|     60|       1|   3.0|2015-09-0

68997

In [ ]:
@timeit
def get_ratings_by_movies_ids(spark, movie_ids):
    query = "select * from clickhouse.movie_lens.ratings where "
    for movie_id in movie_ids:
        query += f"movie_id = {movie_id} or "
    query = query[:-4]
    spark.sql(query)
    df.show()
    return df

df = get_ratings_by_movies_ids(spark, [1, 2])
df.count()

+-------+--------+------+-------------------+
|user_id|movie_id|rating|          timestamp|
+-------+--------+------+-------------------+
|     10|       1|   2.5|2007-01-20 10:53:51|
|     11|       1|   3.0|1996-12-09 05:44:36|
|     17|       1|   4.0|2002-07-22 09:42:31|
|     19|       1|   3.0|2000-11-20 14:14:48|
|     20|       1|   5.0|2019-03-21 23:03:50|
|     23|       1|   3.0|2019-10-04 18:52:12|
|     24|       1|   4.0|1999-12-16 21:15:45|
|     28|       1|   4.0|2000-06-20 01:08:47|
|     33|       1|   5.0|2008-11-11 07:07:49|
|     34|       1|   4.0|2007-11-11 07:43:38|
|     36|       1|   3.0|2015-01-10 04:42:19|
|     37|       1|   1.0|2001-06-17 04:49:45|
|     43|       1|   5.0|1996-06-09 19:14:09|
|     46|       1|   4.0|2015-10-04 16:36:55|
|     51|       1|   3.5|2006-06-27 02:14:04|
|     54|       1|   4.0|1997-05-20 18:36:51|
|     57|       1|   4.0|1997-01-07 03:07:11|
|     59|       1|   4.0|2002-08-27 03:57:42|
|     60|       1|   3.0|2015-09-0

68997

### Complicated queries

In [18]:
@timeit
def get_avg_recent_ratings(spark):
    query = f"with recent_rating as (select r.user_id, r.movie_id, r.rating, r.timestamp from clickhouse.movie_lens.ratings r where r.timestamp >= (select max(timestamp) - interval 1 year from clickhouse.movie_lens.ratings)), user_activity as (select user_id, count(*) as total_rated_movie from recent_rating group by user_id) select user_id from user_activity order by total_rated_movie desc limit 10"
    
    df = spark.sql(query)
    # df = spark.sql(f"select * from clickhouse.movie_lens.ratings where movie_id = {movie_id}")
    # df = spark.sql(f"select * from clickhouse.movie_lens.ratings where user_id = {user_id}")
    df.show()
    return df

df = get_avg_recent_ratings(spark)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
@timeit
def get_avg_recent_ratings(spark):
    query = f"with recent_rating as (select r.user_id, r.movie_id, r.rating, r.timestamp from clickhouse.movie_lens.ratings r where r.timestamp >= (select max(timestamp) - interval 1 year from clickhouse.movie_lens.ratings)), user_activity as (select user_id, count(*) as total_rated_movie from recent_rating group by user_id) select user_id from user_activity order by total_rated_movie desc"
    
    df = spark.sql(query)
    # df = spark.sql(f"select * from clickhouse.movie_lens.ratings where movie_id = {movie_id}")
    # df = spark.sql(f"select * from clickhouse.movie_lens.ratings where user_id = {user_id}")
    df.show(10)
    return df

df = get_avg_recent_ratings(spark)

+-------+
|user_id|
+-------+
| 103013|
| 108412|
|  87324|
| 161180|
|   1668|
|  98875|
| 159164|
|  22270|
|  57389|
|  43302|
+-------+
only showing top 10 rows

get_avg_recent_ratings took: 253.95917391777039 sec


## Use Apache Spark vs Pandas for data processing

In [19]:
import pandas as pd
# import chdb.dataframe as cdf
import numpy as np

### Dataframe operation

In [ ]:
# # join in query vs join after loaded into dataframe

# @timeit
# def join_movies_ratings_query(spark):
#     query = f"select m.title, m.genres from clickhouse.movie_lens.movies m join clickhouse.movie_lens.ratings r on m.movie_id = r.movie_id"
#     df = spark.sql(query)
#     df.show(20)
#     return df

# def join_movies_ratings_df(spark):
#     df_movies = spark.sql("select * from clickhouse.movie_lens.movies")
#     df_ratings = spark.sql("select * from clickhouse.movie_lens.ratings")
#     df = df_movies.join(df_ratings, df_movies.movie_id == df_ratings.movie_id)
#     df.show(20)
#     return df

In [43]:
@timeit
def load_and_join_spark():
    df_ratings = spark.read.format("csv").option("header", "true").load("dataset/ml-32m/ratings.csv")
    df_movies = spark.read.format("csv").option("header", "true").load("dataset/ml-32m/movies.csv")

    df_joined = df_movies.join(df_ratings, df_movies.movieId == df_ratings.movieId).select(df_movies.title, df_movies.genres, df_ratings.userId, df_ratings.rating, df_ratings.timestamp, df_ratings.movieId)

    df_joined.show(20)
    return df_joined

df_joined_spark = load_and_join_spark()

+--------------------+--------------------+------+------+---------+-------+
|               title|              genres|userId|rating|timestamp|movieId|
+--------------------+--------------------+------+------+---------+-------+
|Sense and Sensibi...|       Drama|Romance|     1|   4.0|944249077|     17|
|Leaving Las Vegas...|       Drama|Romance|     1|   1.0|944250228|     25|
|City of Lost Chil...|Adventure|Drama|F...|     1|   2.0|943230976|     29|
|Shanghai Triad (Y...|         Crime|Drama|     1|   5.0|944249077|     30|
|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|     1|   5.0|943228858|     32|
|         Babe (1995)|      Children|Drama|     1|   2.0|943228491|     34|
|Dead Man Walking ...|         Crime|Drama|     1|   1.0|944249008|     36|
|White Balloon, Th...|      Children|Drama|     1|   5.0|944248943|     80|
|   Braveheart (1995)|    Action|Drama|War|     1|   3.0|943231119|    110|
|  Taxi Driver (1976)|Crime|Drama|Thriller|     1|   5.0|944249008|    111|
| Crimson Ti

In [30]:
# %pip install dask

In [29]:
# import dask.dataframe as dd

In [28]:
# @timeit
# def load_and_join_dask():
#     df_movies = dd.read_csv("dataset/ml-32m/movies.csv")
#     df_ratings = dd.read_csv("dataset/ml-32m/ratings.csv")
#     df = dd.merge(df_movies, df_ratings, left_on="movieId", right_on="movieId")
#     print(df.head(20))
#     return df

# load_and_join_dask()

In [42]:
# pandas loads and joins df > 4 times slower than spark for ddataset of 32M records
@timeit
def load_and_join_pandas():
    df_ratings = pd.read_csv("dataset/ml-32m/ratings_chunk_2.csv")
    df_movies = pd.read_csv("dataset/ml-32m/movies.csv")

    df_joined = pd.merge(df_movies, df_ratings, on="movieId")

    print(df_joined.head(20))
    return df_joined

df_joined_pd = load_and_join_pandas()

    movieId             title                                       genres  \
0         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
5         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
6         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
7         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
8         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
9         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
10        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
11        1  Toy Story (1995)  Adventure|Animation|Children|Come

In [50]:
@timeit
def basic_opts_pd(df_pd):
    df_pd = df_pd[df_pd["rating"] > 2].groupby("movieId").agg({"rating": "mean"})
    print(df_pd.head(20))
    return df_pd

basic_opts_pd(df_joined_pd)

           rating
movieId          
1        4.036510
2        3.551806
3        3.494055
4        3.459220
5        3.439618
6        3.968984
7        3.610801
8        3.478202
9        3.375996
10       3.609711
11       3.813545
12       3.387845
13       3.653322
14       3.663689
15       3.403010
16       3.952247
17       4.103339
18       3.697835
19       3.418667
20       3.319239
basic_opts_pd took: 1.195014238357544 sec


,rating
movieId,
1,4.036510
2,3.551806
3,3.494055
4,3.459220
5,3.439618
...,...
292609,4.000000
292611,4.000000
292615,3.000000


### Create a new column named "Year"

In [38]:
from pyspark.sql.functions import substring
from pyspark.sql.functions import col


@timeit
def extract_year_from_title(df):
    df = df.withColumn("year", substring("title", -5, 4)).filter(col("year").rlike("^[12]")).withColumn("year", col("year").cast("int"))
    df.show(20)
    return df

df_joined_spark_with_year = extract_year_from_title(df_joined_spark)

+-------+--------------------+--------------------+------+-------+------+---------+----+
|movieId|               title|              genres|userId|movieId|rating|timestamp|year|
+-------+--------------------+--------------------+------+-------+------+---------+----+
|     17|Sense and Sensibi...|       Drama|Romance|     1|     17|   4.0|944249077|1995|
|     25|Leaving Las Vegas...|       Drama|Romance|     1|     25|   1.0|944250228|1995|
|     29|City of Lost Chil...|Adventure|Drama|F...|     1|     29|   2.0|943230976|1995|
|     30|Shanghai Triad (Y...|         Crime|Drama|     1|     30|   5.0|944249077|1995|
|     32|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|     1|     32|   5.0|943228858|1995|
|     34|         Babe (1995)|      Children|Drama|     1|     34|   2.0|943228491|1995|
|     36|Dead Man Walking ...|         Crime|Drama|     1|     36|   1.0|944249008|1995|
|     80|White Balloon, Th...|      Children|Drama|     1|     80|   5.0|944248943|1995|
|    110|   Bravehear

In [ ]:
@timeit
def extract_year_from_title_pd(df):
    df["year"] = df["title"].str[-5:-1]
    df = df[df["year"].str.match("^[12]\d{3}")].astype({"year": "int"})     # this line takes approx 12 secs
    print(df.head(20))
    return df

df_joined_pd_with_year = extract_year_from_title_pd(df_joined_pd)

    movieId             title                                       genres  \
0         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
5         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
6         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
7         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
8         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
9         1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
10        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
11        1  Toy Story (1995)  Adventure|Animation|Children|Come

## Train a movie recommender model

### Linear Regression Model

In [ ]:
# recommender = RecommendationEngine(joined_df)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
feature_assembler = VectorAssembler(inputCols=["userId", "movieId"], outputCol="features")
assembled_df = feature_assembler.transform(joined_df)
(training_data, test_data) = assembled_df.randomSplit([0.8, 0.2])

model = LinearRegression(featuresCol="features", labelCol="rating", predictionCol="prediction")
lr_model = model.fit(training_data)

In [ ]:
# make predictions
predictions = lr_model.transform(test_data)

# show predictions
predictions.select("features", "rating", "prediction").show()

# evaluate the model using R2 and RMSE
evaluator = RegressionEvaluator(labelCol="rating", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

evaluator.setMetricName("r2")
r2 = evaluator.evaluate(predictions)
print(f"R2: {r2}")

+------------+------+------------------+
|    features|rating|        prediction|
+------------+------+------------------+
| [19.0,12.0]|   1.0|3.5889894264465667|
| [44.0,12.0]|   1.0|  3.58205064574231|
|[276.0,12.0]|   4.0|3.5176587608068086|
|[571.0,12.0]|   1.0|3.4357811484965803|
| [44.0,12.0]|   1.0|  3.58205064574231|
|[151.0,12.0]|   3.0|3.5523526643280916|
|[276.0,12.0]|   4.0|3.5176587608068086|
|[524.0,12.0]|   1.0| 3.448826056220583|
| [19.0,12.0]|   1.0|3.5889894264465667|
|[217.0,12.0]|   3.0|3.5340342832688543|
|[274.0,12.0]|   3.5| 3.518213863263149|
|[524.0,12.0]|   1.0| 3.448826056220583|
|[599.0,12.0]|   1.5| 3.428009714107813|
|[276.0,12.0]|   4.0|3.5176587608068086|
|[288.0,12.0]|   2.0|3.5143281460687654|
|[571.0,12.0]|   1.0|3.4357811484965803|
|[217.0,12.0]|   3.0|3.5340342832688543|
|[288.0,12.0]|   2.0|3.5143281460687654|
|[294.0,12.0]|   1.0| 3.512662838699744|
|[350.0,12.0]|   3.0| 3.497119969922209|
+------------+------+------------------+
only showing top

### ALS

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

ratings = joined_df.select("userId", "movieId", "rating")
(training, test) = ratings.randomSplit([0.8, 0.2])

# set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# evaluate
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

Root-mean-square error = 0.546270888146419


In [ ]:
print(userRecs.show())

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3925, 6.7853904...|
|     2|[{74754, 7.789781...|
|     3|[{48322, 6.951864...|
|     4|[{34338, 7.271497...|
|     5|[{5034, 6.932786}...|
|     6|[{971, 7.036011},...|
|     7|[{4678, 9.650388}...|
|     8|[{5055, 7.2378206...|
|     9|[{106100, 7.42635...|
|    10|[{2693, 8.148736}...|
|    11|[{3200, 6.935401}...|
|    12|[{26258, 8.614412...|
|    13|[{2148, 7.473914}...|
|    14|[{2906, 11.770087...|
|    15|[{1734, 7.8901587...|
|    16|[{3925, 5.546985}...|
|    17|[{1658, 6.442111}...|
|    18|[{3200, 5.1955605...|
|    19|[{5666, 5.1309776...|
|    20|[{179819, 7.15204...|
+------+--------------------+
only showing top 20 rows

None


In [ ]:
userSubsetRecs.show()
users.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   463|[{79224, 7.039547...|
|   496|[{34338, 9.542832...|
|   148|[{4450, 7.3070965...|
+------+--------------------+

+------+
|userId|
+------+
|   463|
|   496|
|   148|
+------+



In [ ]:
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[{569, 5.2485666}...|
|   3175|[{258, 6.324589},...|
|   2366|[{494, 7.989951},...|
+-------+--------------------+

